# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: Lora 
* Model: GPT-2
* Evaluation approach: Trainer 
* Fine-tuning dataset: imdb (positive or negative comment)

In [1]:
import torch

# CUDA CHECK
# Set the device as GPU, MPS, or CPU according to availability
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

print(f"Using device: {device}")

Using device: cuda


## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [5]:
! pip install -q "datasets==2.15.0"

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Defining tokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

# Defining model
model = AutoModelForSequenceClassification.from_pretrained("gpt2", num_labels = 2).to(device)
model.config.pad_token_id = tokenizer.pad_token_id

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
from datasets import load_dataset

dataset = load_dataset("imdb")

dataset

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [4]:
splits = ["train", "test"]

ds = {split: ds for split, ds in zip(splits, load_dataset("imdb", split=splits))}
# taking only 6000 text and labels from whole dataset randomly
for split in splits:
    ds[split] = ds[split].shuffle(seed=42).select(range(6000))

In [5]:
ds

{'train': Dataset({
     features: ['text', 'label'],
     num_rows: 6000
 }),
 'test': Dataset({
     features: ['text', 'label'],
     num_rows: 6000
 })}

In [6]:
def preprocess_function(db):
    tok_inputs = tokenizer(db["text"], padding = True, truncation = True)
    tok_inputs["label"] = db["label"]
    return tok_inputs

train_set = ds["train"]
test_set = ds["test"]

train_subset = train_set.map(preprocess_function, batched = True)
test_subset = train_set.map(preprocess_function, batched = True)

Map:   0%|          | 0/6000 [00:00<?, ? examples/s]

Map:   0%|          | 0/6000 [00:00<?, ? examples/s]

In [10]:
import numpy as np
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir="./data/imdb_model",
        # Set the learning rate
        learning_rate = 1e-3,
        # Set the per device train batch size and eval batch size
        per_device_train_batch_size = 5,
        per_device_eval_batch_size = 5,
        # Evaluate and save the model after each epoch
        evaluation_strategy = "epoch",
        save_strategy = "epoch",
        num_train_epochs=1,
        weight_decay=0.01,
        load_best_model_at_end=True,
    ),
    train_dataset=train_subset,
    eval_dataset=test_subset,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

trainer.train()

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.719800,0.695852,0.502500


TrainOutput(global_step=1200, training_loss=0.7465193176269531, metrics={'train_runtime': 1765.9144, 'train_samples_per_second': 3.398, 'train_steps_per_second': 0.68, 'total_flos': 3146432643072000.0, 'train_loss': 0.7465193176269531, 'epoch': 1.0})

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [19]:
from peft import LoraConfig, get_peft_model, TaskType
# Setup loraConfig
config = LoraConfig(task_type = TaskType.SEQ_CLS, inference_mode = False, r = 8, lora_alpha = 32, lora_dropout = 0.1)
# setup our model which will make predictions with num_labels equals to number of outcomes that model may come
model = AutoModelForSequenceClassification.from_pretrained("gpt2", num_labels = 2).to(device)
model.config.pad_token_id = tokenizer.pad_token_id
# connecting lora to model
peft_model = get_peft_model(model, config)
peft_model.print_trainable_parameters()

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 297,984 || all params: 124,737,792 || trainable%: 0.23888830740245906


In [18]:
#Out of the all GPT-2 model only 0.23% will be trained on

In [20]:
# TRAIN THE MODEL

import numpy as np
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

# peft_model because we are training on PEFT
trainer = Trainer(
    model=peft_model,
    args=TrainingArguments(
        output_dir="./data/lora-gpt-model",
        # Set the learning rate
        learning_rate = 1e-3,
        # Set the per device train batch size and eval batch size
        per_device_train_batch_size = 5,
        per_device_eval_batch_size = 5,
        # Evaluate and save the model after each epoch
        evaluation_strategy = "epoch",
        save_strategy = "epoch",
        num_train_epochs=1,
        weight_decay=0.01,
        load_best_model_at_end=True,
    ),
    train_dataset=train_subset,
    eval_dataset=test_subset,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.363500,0.228733,0.932500


TrainOutput(global_step=1200, training_loss=0.4323532994588216, metrics={'train_runtime': 1788.0128, 'train_samples_per_second': 3.356, 'train_steps_per_second': 0.671, 'total_flos': 3146489266176000.0, 'train_loss': 0.4323532994588216, 'epoch': 1.0})

In [40]:
# Saving the pretrained model
peft_model.save_pretrained("gpt-peft")

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [26]:
from peft import AutoPeftModelForSequenceClassification

inf_model = AutoPeftModelForSequenceClassification.from_pretrained("gpt-peft").to(device)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [35]:
def predict(example: str) -> str:

    inputs = tokenizer(example, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = inf_model(**inputs)
        logits = outputs.logits

    probabilities = torch.nn.functional.softmax(logits, dim=1)
    predicted_class_id = probabilities.argmax().item()
    if (predicted_class_id == 0):
        predicted_label = "Negative review"
    else:
        predicted_label = "Positive review"

    return predicted_label

input_value = "The moview is quite interesting in the beggining"
prediction_value = predict(input_value)
print(f"Sentence: '{input_value}'\nPredicted label: {prediction_value}")

Sentence: 'The moview is quite interesting in the beggining'
Predicted label: Positive review


In [36]:
input_value = "To be honest the movie was bad because of characters. Also, it was quite boring to watch this movie"
prediction_value = predict(input_value)
print(f"Sentence: '{input_value}'\nPredicted label: {prediction_value}")

Sentence: 'To be honest the movie was bad because of characters. Also, it was quite boring to watch this movie'
Predicted label: Negative review


In [37]:
input_value = "Recommend this to watch with family members especially with kids"
prediction_value = predict(input_value)
print(f"Sentence: '{input_value}'\nPredicted label: {prediction_value}")

Sentence: 'Recommend this to watch with family members especially with kids'
Predicted label: Positive review


## Results

Overall we can see that prior fine-tuning the model results were not quite good as Accuracy was around 0.5 and validation loss 0.69.

However situation changed when we trained and fine-tuned our model in the new data using PEFT approach. As model were trained on a new data and specific task with 0.23 percent of new trainable data and also last layer were changed only 2 outputs. So results that we got is Accuracy: 0.93 and Validation loss: 0.22. 

In our examples we can see that our model is quite good in predictions whether review is positive or negative based on the text that users wrote.